In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import fasttext
import fasttext.util
from sklearn.decomposition import PCA



df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')

#fasttext.util.download_model('en', if_exists='ignore')
en_model = fasttext.load_model('/kaggle/input/document-represenataion-with-fasttext/cc.en.300.bin')

df['excerpt_feature'] = df['excerpt'].fillna("").str.replace("\n", "").apply(lambda x: en_model.get_sentence_vector(x))
print(df['excerpt_feature'].values[0])



In [ ]:

X = np.array(df['excerpt_feature'].values.tolist()).astype(np.float32)
print(X)

pca = PCA(n_components=2)
pca.fit(X)
Xd = pca.transform(X)

plt.scatter(Xd.T[0], Xd.T[1])


In [ ]:
plt.scatter(Xd.T[0], Xd.T[1], s=100, c=df['target'].values, cmap='Blues')
plt.colorbar()

In [ ]:
X = np.array(df['excerpt_feature'].values.tolist()).astype(np.float32)
print(X)

pca = PCA(n_components=3)
pca.fit(X)
Xd = pca.transform(X)
print(Xd)

# Import dependencies
import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
    x=Xd.T[0],  # <-- Put your data instead
    y=Xd.T[1],  # <-- Put your data instead
    z=Xd.T[2],  # <-- Put your data instead
    mode='markers',
    marker={
        'size': 4,
        'opacity': 0.8,
        'color': df['target'].values,
        'colorscale': 'Blues',
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
    width=1500,
    height=1300
)

data = [trace]

plot_figure = go.Figure(
    data=data,
    layout=layout,
)

# Render the plot.
plotly.offline.iplot(plot_figure)

In [ ]:
import seaborn as sns
X = np.array(df['excerpt_feature'].values.tolist()).astype(np.float32)
print(X)

pca = PCA(n_components=1)
pca.fit(X)
Xd = pca.transform(X)

plt.scatter(np.arange(len(Xd)), Xd*20)


In [ ]:
plt.scatter(np.arange(len(df['target'].values)), df['target'].values)

In [ ]:
print(len(df[df['target'] > 0]))
print(len(df['target']))

In [ ]:
Xd.reshape(-1,)

In [ ]:
np.dot(Xd.reshape(-1,), df['target'].values) / (np.linalg.norm(Xd.reshape(-1,)) * np.linalg.norm(df['target'].values))

In [ ]:
plt.scatter(Xd.reshape(-1,)*20, df['target'].values)

In [ ]:
x = Xd.reshape(-1,)
y = df['target'].values
np.corrcoef(x, y)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, y)
reg.score(X, y)

In [ ]:
import torch
import numpy as np
import torch.utils.data as data_utils

In [ ]:
df['excerpt_feature'][0].shape

In [ ]:
feature = np.array(df['excerpt_feature'].values.tolist()).astype(np.float32)
train = data_utils.TensorDataset(torch.tensor(feature), torch.tensor(df['target'].values.astype(np.float32)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


# NOTE: pretty simple NN model
class SimpleNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(300, 200)
        self.l3 = nn.Linear(200, 100)
        self.l4 = nn.Linear(100, 50)
        self.l5 = nn.Linear(50, 10)
        self.l7 = nn.Linear(10, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.relu(self.l3(x))
        x = self.relu(self.l4(x))
        x = self.relu(self.l5(x))
        x = self.l7(x)
        return x
    

In [ ]:
torch.manual_seed(1)

dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_model = SimpleNN()
print(train_model)

train_model = train_model.to(dev)

optimizer = torch.optim.Adam(train_model.parameters(), lr=0.01)

loss_func = nn.MSELoss()

In [ ]:
BATCH_SIZE = 64
EPOCH = 10000

loader = torch.utils.data.DataLoader(
    dataset=train, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, y)
reg.score(X, y)

In [ ]:
test_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')

In [ ]:
test_df['excerpt_feature'] = test_df['excerpt'].fillna("").str.replace("\n", "").apply(lambda x: en_model.get_sentence_vector(x))

In [ ]:
test_X = X = np.array(df['excerpt_feature'].values.tolist()).astype(np.float32)
test_df['target'] = test_df['excerpt_feature'].apply(lambda x: reg.predict(x.reshape(1, -1))[0])

In [ ]:
test_df['target']

In [ ]:
submission = pd.DataFrame({'id': test_df['id'],'target': test_df['target']})
submission

In [ ]:
submission.to_csv('submission.csv',index=False)